In [1]:
import os
import numpy as np
import pandas as pd

import riskflow_jupyter

In [2]:
rundate = '2022-07-07'
if os.name=='nt':
    path = os.path.join('U:\\CVA_JSON', rundate)
    path_transform={
        '\\\\ICMJHBMVDROPPRD\\AdaptiveAnalytics\\Inbound\\MarketData':
        '\\\\ICMJHBMVDROPUAT\\AdaptiveAnalytics\\Inbound\\MarketData'}
else:
    path = os.path.join('/media/vretiel/3EFA4BCDFA4B7FDF/Media/Data/crstal/CVA_JSON', rundate)
    path_transform={
            '//ICMJHBMVDROPPRD/AdaptiveAnalytics/Inbound/MarketData': 
            '/media/vretiel/3EFA4BCDFA4B7FDF/Media/Data/crstal/CVA_JSON'}
    
cx = riskflow_jupyter.rf.Context(
    path_transform=path_transform,
    file_transform={
        'CVAMarketData_Calibrated.dat': 'CVAMarketData_Calibrated_New.json',
        'MarketData.dat': 'MarketData.json'
    })

In [3]:
from riskflow_widgets import Flot, Table, Three, to_json

In [4]:
# cx.load_json(os.path.join(path, 'InputAAJ_CrB_Russellstone_Treasury_ISDA.json'))
# cx.load_json(os.path.join(path, 'InputAAJ_CrB_BNP_Paribas__Paris__ISDA.json'))
cx.load_json(os.path.join(path, 'InputAAJ_CrB_Natixis_ISDA.json'))



In [5]:
cx.config_cache

{'\\\\ICMJHBMVDROPPRD\\AdaptiveAnalytics\\Inbound\\MarketData\\CVAMarketData_Calibrated.dat': <riskflow.adaptiv.AdaptivContext at 0x7f717942bd60>}

In [6]:
import ipywidgets as widgets


In [ ]:
factor = riskflow_jupyter.rf.utils.Factor('FXVol',('GBP','ZAR'))
# factor = riskflow_jupyter.rf.utils.Factor('InterestYieldVol',('CAD_SMILE_ICE',))

vol = riskflow_jupyter.rf.riskfactors.construct_factor( factor,
    cx.current_cfg.params['Price Factors'],
    cx.current_cfg.params['Price Factor Interpolation']
)

In [ ]:
j=riskflow_jupyter.load_table_from_vol(vol)
#to_json(j)
#np.isnan(None)

In [ ]:
# g = np.vstack([x[0], heights]).tolist()
t = Three(description='Test')
t.observe(lambda x:x['new'], 'value')
t.value = to_json(j)

In [ ]:
t

In [ ]:
class CalculationResults(widgets.HBox):
    def __init__(self, description):
        self.description = widgets.Label(value=description)
        self.plot = k3d.plot(
            axes=['log(moneyness)', 'expiry', 'vol(\%)'],
            # menu_visibility=False,
            camera_rotate_speed=3.0,
        )
        # set the plot layout to match the table below
        self.plot.layout.max_width = '600px'

        self.mesh = None
        self.points = None
        self.plot_label = None
        self.data = Table(description='', settings=to_json({
            'width': 600, 'height': 300, 'contextMenu': True, 'minSpareRows': 1, 'minSpareCols': 1
        }))

        self.add_button = widgets.Button(
            description='Add Tenor', tooltip='Add a new vol surface for a new tenor')
        self.del_button = widgets.Button(
            description='Remove Tenor', tooltip='Delete vol surface for this tenor')
        self.dropdown = widgets.Combobox(
            description='Tenor:', placeholder='Choose Tenor', options=[],
            ensure_option=False)
        self.dropdown.layout.max_width='360px'
        self.tenor = None

        self.selector = widgets.HBox(children=[self.add_button, self.del_button, self.dropdown])
        self.selector.layout.visibility = 'hidden'
        self.selector.layout.max_width = '600px'

        self.add_button.on_click(self.add_button_clicked)
        self.del_button.on_click(self.del_button_clicked)
        self.dropdown.observe(self.change_selection, 'value')
        self.obj = None

        super().__init__(
            children=[
                self.description,
                widgets.VBox(children=[self.selector, self.plot, self.data])
            ]
        )

    def change_selection(self, change):
        if change.new in self.obj:
            matrix = self.obj[change.new]
            self.data.value = to_json(matrix)
            self.update_plot(matrix)
            self.tenor = self.dropdown.value
            self.plot_label.text = 'Tenor: {}'.format(self.tenor)
            self.dropdown.unobserve(self.change_selection, 'value')
            self.dropdown.value = ''
            self.dropdown.observe(self.change_selection, 'value')

    def add_button_clicked(self, b):
        val = self.dropdown.value
        if val:
            try:
                val = self.dropdown.value
                self.dropdown.options = tuple([str(x) for x in sorted(
                    [float(x) for x in (self.dropdown.options + (val,))])])
                self.tenor = val
                self.plot_label.text = 'Tenor: {}'.format(self.tenor)
                self.obj[val] = json.loads(riskflow_jupyter.rf.fields.default['Surface'])
                matrix = self.obj[val]
                self.data.value = to_json(matrix)
                self.update_plot(matrix)
            except:
                print('could not cast value to float')

    def del_button_clicked(self, b):
        val = self.dropdown.value if self.dropdown.value else self.tenor
        if val in self.dropdown.options:
            self.dropdown.options = tuple([x for x in self.dropdown.options if x != val])
            print('inside - about to del', self.obj.keys())
            del self.obj[val]
            print('inside - about to del', self.obj.keys())
            if self.dropdown.options:
                self.tenor = self.dropdown.options[0]
                matrix = self.obj[self.tenor]
            else:
                self.tenor = ''
                matrix = [[]]

            self.dropdown.value = self.tenor
            print('inside - after del', self.obj.keys())
            self.data.value = to_json(matrix)
            self.update_plot(matrix)
        print("del Button clicked.", val)

    def observe(self, handler, prop, type='change'):

        def make_plot_fn():
            def update_plot(change):
                # now update the table with the new data
                table_obj = json.loads(change.new)
                # store the result if this is a 3d obj
                if self.tenor is not None:
                    self.obj[self.tenor] = table_obj
                # call the original handler with the modified change event
                handler({'name': change.name, 'old': change.old, 'new': self.value,
                         'owner': self, 'type': change.type})
                # check if we have a valid table obj
                if np.all([len(x) > 1 for x in table_obj[1:]]):
                    self.update_plot(table_obj)

            return update_plot

        if self.children:
            # link the observable function to the table widget (self.data)
            self.data.observe(make_plot_fn(), prop, type)
        else:
            super().observe(handler, prop, type)

    @staticmethod
    def make_faces_vectorized1(Nr, Nc):

        out = np.empty((Nr - 1, Nc - 1, 2, 3), dtype=int)

        r = np.arange(Nr * Nc).reshape(Nr, Nc)

        out[:, :, 0, 0] = r[:-1, :-1]
        out[:, :, 1, 0] = r[:-1, 1:]
        out[:, :, 0, 1] = r[:-1, 1:]

        out[:, :, 1, 1] = r[1:, 1:]
        out[:, :, :, 2] = r[1:, :-1, None]

        out.shape = (-1, 3)
        return out

    @staticmethod
    def interpolate_surface(json_list):
        moneyness = json_list[0][1:]
        e = []
        for p in json_list[1:]:
            e.extend([[m, p[0], v] for m, v in zip(moneyness, p[1:]) if v is not None])

        surface = np.array(e)
        expiry = [p[0] for p in json_list[1:]]
        return e, expiry, moneyness, np.array([np.interp(
            moneyness, surface[surface[:, 1] == x][:, 0], surface[surface[:, 1] == x][:, 2]) for x in expiry])

    def update_plot(self, json_list):
        raw_points, expiry, moneyness, vol = Three.interpolate_surface(json_list)
        raw_vertices = np.array(raw_points, dtype=np.float32)

        if min(moneyness) > 0:
            scale = 2 / np.log(2)
            m = scale * np.log(moneyness)
            raw_vertices[:, 0] = scale * np.log(raw_vertices[:, 0])
        else:
            m = np.array(moneyness) * 100.0
            raw_vertices[:,0] = 100.0*raw_vertices[:,0]
            self.plot.axes = ['moneyness (bps)', 'expiry', 'vol(\%)']

        v = vol * 100
        raw_vertices[:, 2] = 100.0 * raw_vertices[:, 2]

        U, V = np.meshgrid(m, expiry)
        vertices = np.dstack([U, V, v]).astype(np.float32).reshape(-1, 3)
        indices = Three.make_faces_vectorized1(*v.shape).astype(np.uint32)
        if self.mesh is None:
            self.mesh = k3d.mesh(
                vertices, indices, flat_shading=False, attribute=v,
                side='double', color_map=k3d.basic_color_maps.Reds, color_range=[v.min(), v.max()]
            )
            self.points = k3d.points(raw_vertices, point_size=0.1, shader='3d')
            self.plot_label = k3d.text2d('Volatility', position=(0, 0))
            # add the plots
            self.plot += self.points
            self.plot += self.mesh
            self.plot += self.plot_label
        else:
            self.points.vertices = raw_vertices
            self.mesh.attribute = v
            self.mesh.color_range = [v.min(), v.max()]
            self.mesh.vertices = vertices
            self.mesh.indices = indices

    @property
    def value(self):
        return to_json(self.obj)

    @value.setter
    def value(self, json_string):
        self.obj = json.loads(json_string)
        if isinstance(self.obj, dict):
            self.selector.layout.visibility = 'visible'
            self.dropdown.options = tuple(self.obj.keys())
            selection = self.dropdown.options[0]
            self.dropdown.value = selection
            matrix = self.obj[selection]
            data_value = to_json(matrix)
        else:
            self.selector.layout.visibility = 'hidden'
            data_value = json_string
            matrix = self.obj

        self.data.value = data_value
        self.update_plot(matrix)


In [ ]:
data

In [ ]:
t.value

In [ ]:
plt_text1.visible=False

In [ ]:
from ipyfilechooser import FileChooser

# Create and display a FileChooser widget
fc = FileChooser('/home/vretiel/miniconda3/')
display(fc)

In [ ]:
cx

In [ ]:
pp = riskflow_jupyter.PortfolioPage(cx)

In [ ]:
pp.main_container

In [ ]:
rfp = riskflow_jupyter.RiskFactorsPage(cx)

In [ ]:
rfp.main_container


In [ ]:
import importlib
importlib.reload(riskflow_jupyter)

In [9]:
cp = riskflow_jupyter.CalculationPage(cx)
cp.main_container

In [14]:
#cx.current_cfg.params['Price Factors']['InterestRate.CHF-OIS']['Curve'].array.dtype
cx.current_cfg.deals['Calculation']

{'Object': 'CreditMonteCarlo',
 'Base_Date': Timestamp('2022-07-07 00:00:00'),
 'Collateral_Algorithm': 'Do Nothing Collateral Algorithm',
 'Currency': 'EUR',
 'Adjust_To_External_MtM': 'No',
 'Base_Time_Grid': <riskflow.utils.Offsets at 0x7f72180f3be0>,
 'Sub_Calculations': [{'Name': 'Bank',
   'Filter': '',
   'Tags': '',
   'Aggregation_Mode': 'Auto',
   'Exposure_View': 'Bank',
   'Settlement_Adjustment': 0,
   'Liquidation_Adjustment': 0},
  {'Name': 'Counterparty',
   'Filter': '',
   'Tags': '',
   'Aggregation_Mode': 'Auto',
   'Exposure_View': 'Counterparty',
   'Settlement_Adjustment': 0,
   'Liquidation_Adjustment': 0}],
 'Scenarios': 100,
 'Antithetic': 'No',
 'Sampling': 'Unstratified',
 'Random_Seed': 1,
 'Output_Detail': 'Quantiles',
 'Use_Dynamic_Dates': 'Yes',
 'Scenario_Time_Grid': <riskflow.utils.Offsets at 0x7f72180f30a0>,
 'Max_Horizon': '',
 'Deflation_Interest_Rate': 'EUR-MASTER',
 'Deterministic_Scenarios': {'Deterministic_Scenarios': 'Off',
  'Scenario_Path': '

In [15]:
calc, output = cx.run_job()

In [ ]:
{k:v for k,v in output.items() if isinstance(v, dict)}


In [ ]:
a=Flot(description='asds', 
       hot_settings=to_json(
           {
               'columns': [{},
                           {"type": "numeric","numericFormat": {"pattern": "0,0.00"}},
                           {"type": "numeric","numericFormat": {"pattern": "0,0.00"}}
                           ],
               'manualColumnResize': True
               }
       ), 
       flot_settings=to_json(
           {
            'xaxis': { 'mode': "time", 'timeBase': "milliseconds"}
           }
        )
    )
a

In [ ]:
#pp.tree_data
# rfp.tree_data[-1]
p=output['Results']['profile']
o = [{'label':c, 'data':[[x,y] for x,y in zip(p.index.astype(np.int64)//1000000, p[c].values)]} for c in p.columns]
a.value=to_json(o)

In [24]:
cp.calculation_fields['CreditMonteCarlo']

{'Base_Date': {'widget': 'DatePicker',
  'description': 'Base Date',
  'value': '',
  'default': ''},
 'Currency': {'widget': 'Text',
  'description': 'Currency',
  'value': 'ZAR',
  'default': 'ZAR'},
 'Base_time_grid': {'widget': 'Text',
  'description': 'Base time grid',
  'value': '0d 2d 1w(1w) 3m(1m) 2y(3m)',
  'default': '0d 2d 1w(1w) 3m(1m) 2y(3m)'},
 'Simulation_Batches': {'widget': 'Integer',
  'description': 'Simulation Batches',
  'value': 1,
  'default': 1},
 'Batch_Size': {'widget': 'Integer',
  'description': 'Batch Size',
  'value': 1024,
  'default': 1024},
 'Random_Seed': {'widget': 'Integer',
  'description': 'Random Seed',
  'value': 5120,
  'default': 5120},
 'Calc_Scenarios': {'widget': 'Dropdown',
  'description': 'Calc Scenarios',
  'value': 'No',
  'values': ['Yes', 'No'],
  'default': 'No'},
 'Generate_Cashflows': {'widget': 'Dropdown',
  'description': 'Generate Cashflows',
  'value': 'Yes',
  'values': ['Yes', 'No'],
  'Output': 'Cashflows',
  'default': 'Yes

In [ ]:
1191189600000, 1658793600000000000

In [ ]:
#[x.milliseconds for x in p.index]
p.index.astype(np.int64)



In [16]:
output['Results'].keys()

dict_keys(['mtm', 'cva', 'profile'])

In [17]:
calc, output = cx.Base_Valuation()

In [22]:
output['Results']['mtm']

,Reference,Object,Deal Currency,Value,Acquirer,Portfolio,Instrument,SalesTeam,Parent
0,CrB_Natixis_ISDA,NettingCollateralSet,None,-111615.260571,,,,,NaN
1,18761421,StructuredDeal,USD,8386.381880,IBL Forex Trading,CDT_BASIS_SWAP,Swap,None,CrB_Natixis_ISDA
2,USDIRSF-LI230929-2409303085,CFFixedInterestListDeal,USD,-136083.944991,NaN,NaN,NaN,NaN,18761421
3,USDIRSF-LI230929-2409303085,CFFloatingInterestListDeal,USD,144470.326871,NaN,NaN,NaN,NaN,18761421
4,22460644,StructuredDeal,USD,-39453.256610,IBL Forex Trading,CDT_BASIS_SWAP,Swap,None,CrB_Natixis_ISDA
5,USDIRSF-LI190708-220708132,CFFixedInterestListDeal,USD,-49263.311688,NaN,NaN,NaN,NaN,22460644
6,USDIRSF-LI190708-220708132,CFFloatingInterestListDeal,USD,9810.055078,NaN,NaN,NaN,NaN,22460644
7,22833355,StructuredDeal,USD,-160139.284679,IBL Forex Trading,CDT_BASIS_SWAP,Swap,None,CrB_Natixis_ISDA
8,USDIRSF-LI180725-250725156,CFFixedInterestListDeal,USD,297285.654415,NaN,NaN,NaN,NaN,22833355
9,USDIRSF-LI180725-250725156,CFFloatingInterestListDeal,USD,-457424.939094,NaN,NaN,NaN,NaN,22833355


In [ ]:
#rfp.right_container.children[0].children[4].plot.layout.max_width='600px'
dir(rfp.right_container.children[0].children[2])

In [ ]:
rfp.right_container.children[0].children[2].dropdown.layout.max_width='360px'
#rfp.right_container.children[0].children[2].selector.layout.max_width='600px'
rfp.right_container.children[0].children[2].selector